# Imports

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import ast

# Load result from benchmark

In [4]:
def read_file(file_path):
    result = pd.read_csv(file_path, sep=';')
    
    list_classes = ['classes_true', 'classes_pred', 'bounding_boxes', 'transcript', 'rec_pred', 'final_pred', 'suggestions']
    
    for x in list_classes:
        result[x] = result[x].apply(ast.literal_eval)
    
    
    return result

In [5]:
file_path = "../benchmark-results/benchmark-result.csv"

result = read_file(file_path)

result.head()

ParserError: Error tokenizing data. C error: Expected 624 fields in line 4, saw 804


In [ ]:
def get_image(img_path):
    return cv.imread(img_path)

def convert_wordlist_to_string(list_of_words):
        nbr_words = len(list_of_words)
        text = ""
        
        if nbr_words > 0:
            text += list_of_words[0]
        
            for i in range(1, nbr_words):
                text += (" " + list_of_words[i])

        return text
    
###################################################################  
def display_example(result):
    
    if (len(result) == 0):
        return
    
    sample = result.sample()
    
    img = get_image(sample.img_path.values[0])
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
    transcript_string = convert_wordlist_to_string(sample.transcript.values[0])
    rec_pred_string = convert_wordlist_to_string(sample.rec_pred.values[0])
    final_pred_string = convert_wordlist_to_string(sample.final_pred.values[0])
    
    plt.imshow(img)
    plt.show()
    print("Transcript:\n", transcript_string)
    print("\nRecognition Pred:\n", rec_pred_string)
    print("\nFinal Pred:\n", final_pred_string)
    
###################################################################    
def evaluate_result(result):
    final_error = round(result['final_error'].mean() * 100, 3)
    rec_error = round(result['rec_error'].mean() * 100, 3)
    det_inf = round(result['det_inf'].mean(), 3)
    rec_inf = round(result['rec_inf'].mean(), 3)
    pp_inf = round(result['pp_inf'].mean(), 3)
    total_inf = round(result['total_inf'].mean(), 3)
    sc_precision = round(result['sc_precision'].mean() * 100, 3)
    sc_recall = round(result['sc_recall'].mean() * 100, 3)
    sc_f1 = round(result['sc_f1'].mean() * 100, 3)

    print("""
          -----------------------------------------------
          Original Recognition error: {} %
          Final recognition error:    {} %\n
          -----------------------------------------------
          Detection inference time:   {} ms / word
          Recognition inference time: {} ms / word
          Postprocess inference time: {} ms / word
          Total inference time:       {} ms / word\n
          -----------------------------------------------
          Spellcheck precision: {} %
          Spellcheck recall:    {} %
          Spellcheck F1:        {} %\n
    """.format(rec_error, final_error, det_inf, rec_inf, pp_inf, total_inf, sc_precision, sc_recall, sc_f1))
    
###################################################################     
def display_incorrect_words(result):
    for form_result in result.itertuples():
        
        # Extract the found incorrect words.
        recognized_words = np.array(form_result.final_pred)
        classes_pred = np.array(form_result.classes_pred)
        found_incorrect_words = recognized_words[classes_pred == 1]
        
        # Extract the correct incorrect words.
        transcript = np.array(form_result.transcript)
        classes_true = np.array(form_result.classes_true)
        true_incorrect_words = transcript[classes_true == 1]
        
        # Prints
        print("######################################################")
        print("Input image: ", form_result.img_path.split("\\")[-1])
        print("\nFound incorrect words:\n", found_incorrect_words)
        print("\nTrue incorrect words:\n", true_incorrect_words)
        
        print("\n")
    

In [ ]:
evaluate_result(result)

In [ ]:
display_example(result)

In [ ]:
display_incorrect_words(result)